# Hyperparameter Tuning with Optuna - RunPod GPU

Uses Optuna to find optimal PPO hyperparameters.

**Search Space:**
- learning_rate: 1e-5 to 1e-3 (log scale)
- n_steps: [1024, 2048, 4096]
- batch_size: [32, 64, 128]
- n_epochs: [5, 10, 15]
- gamma: 0.95 to 0.999
- ent_coef: 0.001 to 0.1 (log scale)
- net_arch_size: [64, 128, 256]

**Strategy:**
- 300k timesteps per trial (vs 1.5M for final training)
- ~10 min per trial
- 25 trials = ~4 hours
- Pruning enabled to skip bad trials early

## 1. Setup

In [ ]:
# Check GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU only'}")
if torch.cuda.is_available():
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Clone/update repository
import os
if not os.path.exists('/workspace/enhanced-rl-portfolio'):
    !git clone https://github.com/nimeshk03/enhanced-rl-portfolio.git /workspace/enhanced-rl-portfolio
else:
    !cd /workspace/enhanced-rl-portfolio && git pull

os.chdir('/workspace/enhanced-rl-portfolio')
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install dependencies
!pip install -q stable-baselines3[extra] gymnasium pandas numpy pyyaml tensorboard optuna

In [ ]:
# Check data files
import os
os.makedirs('data', exist_ok=True)

price_exists = os.path.exists('data/processed_data.csv')
sentiment_exists = os.path.exists('data/historical_sentiment_complete.csv')

if price_exists and sentiment_exists:
    print("Data files found!")
else:
    print("Missing data files - upload before continuing:")
    if not price_exists: print("  - data/processed_data.csv")
    if not sentiment_exists: print("  - data/historical_sentiment_complete.csv")

## 2. Run Hyperparameter Search

In [ ]:
# Configuration
N_TRIALS = 25          # Number of trials (25 recommended, ~4 hours)
TIMESTEPS = 300_000    # Timesteps per trial (reduced for speed)

print(f"Trials: {N_TRIALS}")
print(f"Timesteps per trial: {TIMESTEPS:,}")
print(f"Estimated time: ~{N_TRIALS * 10} minutes ({N_TRIALS * 10 / 60:.1f} hours)")

In [ ]:
# Run hyperparameter search
!python -m src.experiments.hyperparameter_search --n-trials {N_TRIALS} --timesteps {TIMESTEPS}

## 3. View Results

In [ ]:
import json
import pandas as pd

# Load best hyperparameters
with open('experiments/optuna_study/best_hyperparameters.json', 'r') as f:
    best = json.load(f)

print("="*60)
print("BEST HYPERPARAMETERS")
print("="*60)
print(f"Best Sharpe: {best['best_sharpe']:.4f}")
print(f"\nParameters:")
for k, v in best['best_params'].items():
    print(f"  {k}: {v}")

if 'best_metrics' in best:
    print(f"\nAdditional Metrics:")
    for k, v in best['best_metrics'].items():
        if isinstance(v, float):
            print(f"  {k}: {v:.4f}")
        else:
            print(f"  {k}: {v}")

In [ ]:
# View all trials
df = pd.read_csv('experiments/optuna_study/all_trials.csv')

print("Top 10 Trials:")
print("="*60)

top_10 = df.nlargest(10, 'value')[['number', 'value', 'params_learning_rate', 'params_ent_coef', 'params_net_arch_size']]
top_10.columns = ['Trial', 'Sharpe', 'LR', 'Ent Coef', 'Net Size']
print(top_10.to_string(index=False))

In [ ]:
# Visualize results
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Sharpe over trials
ax = axes[0, 0]
ax.plot(df['number'], df['value'], 'b-', alpha=0.5)
ax.scatter(df['number'], df['value'], c='blue', alpha=0.5)
ax.axhline(y=df['value'].max(), color='r', linestyle='--', label=f'Best: {df["value"].max():.3f}')
ax.set_xlabel('Trial')
ax.set_ylabel('Sharpe Ratio')
ax.set_title('Sharpe Ratio Over Trials')
ax.legend()

# Learning rate vs Sharpe
ax = axes[0, 1]
ax.scatter(df['params_learning_rate'], df['value'], alpha=0.6)
ax.set_xscale('log')
ax.set_xlabel('Learning Rate')
ax.set_ylabel('Sharpe Ratio')
ax.set_title('Learning Rate vs Sharpe')

# Entropy coef vs Sharpe
ax = axes[1, 0]
ax.scatter(df['params_ent_coef'], df['value'], alpha=0.6)
ax.set_xscale('log')
ax.set_xlabel('Entropy Coefficient')
ax.set_ylabel('Sharpe Ratio')
ax.set_title('Entropy Coef vs Sharpe')

# Network size vs Sharpe
ax = axes[1, 1]
for size in [64, 128, 256]:
    mask = df['params_net_arch_size'] == size
    ax.scatter([size]*mask.sum(), df.loc[mask, 'value'], alpha=0.6, label=f'{size}')
ax.set_xlabel('Network Size')
ax.set_ylabel('Sharpe Ratio')
ax.set_title('Network Size vs Sharpe')

plt.tight_layout()
plt.savefig('experiments/optuna_study/optimization_plots.png', dpi=150)
plt.show()
print("Saved to experiments/optuna_study/optimization_plots.png")

## 4. Train Final Model with Best Hyperparameters

In [ ]:
# Train full model with best hyperparameters
import json
import os
import sys
from datetime import datetime
import numpy as np
import torch

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

sys.path.insert(0, '.')
from src.env.enhanced_portfolio_env import EnhancedPortfolioEnv
from src.data.enhanced_processor import EnhancedDataProcessor, ProcessorConfig

# Load best params
with open('experiments/optuna_study/best_hyperparameters.json', 'r') as f:
    best = json.load(f)

params = best['best_params']
print("Training with best hyperparameters:")
for k, v in params.items():
    print(f"  {k}: {v}")

In [ ]:
# Load data
def prepare_df_for_env(df):
    df = df.copy()
    df = df.sort_values(['date', 'tic']).reset_index(drop=True)
    dates = sorted(df['date'].unique())
    date_to_day = {date: i for i, date in enumerate(dates)}
    df['day'] = df['date'].map(date_to_day)
    return df.set_index('day')

processor = EnhancedDataProcessor(
    price_path='data/processed_data.csv',
    sentiment_path='data/historical_sentiment_complete.csv',
    config=ProcessorConfig(normalize_features=True, normalization_window=60),
)

train_df, test_df = processor.get_train_test_split(
    train_end="2024-06-30",
    test_start="2024-07-01",
    test_end="2025-11-30",
)

feature_info = processor.get_feature_info()
train_df_indexed = prepare_df_for_env(train_df)
test_df_indexed = prepare_df_for_env(test_df)

print(f"Train: {len(train_df)} records")
print(f"Test: {len(test_df)} records")

In [ ]:
# Create environment and train
TOTAL_TIMESTEPS = 1_500_000

env_config = {
    "hmax": 100,
    "initial_amount": 100000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "reward_scaling": 1e-4,
    "sentiment_reward_weight": 0.0,
}

train_env = EnhancedPortfolioEnv(
    df=train_df_indexed,
    stock_dim=feature_info['n_tickers'],
    tech_indicator_list=feature_info['tech_indicators'],
    sentiment_feature_list=feature_info['sentiment_features'],
    include_sentiment=True,
    normalize_obs=False,
    print_verbosity=0,
    **env_config,
)

train_env_wrapped = DummyVecEnv([lambda: Monitor(train_env)])

# Create model with best params
net_size = params['net_arch_size']
policy_kwargs = {"net_arch": dict(pi=[net_size, net_size], vf=[net_size, net_size])}

model = PPO(
    "MlpPolicy",
    train_env_wrapped,
    learning_rate=params['learning_rate'],
    n_steps=params['n_steps'],
    batch_size=params['batch_size'],
    n_epochs=params['n_epochs'],
    gamma=params['gamma'],
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=params['ent_coef'],
    vf_coef=params['vf_coef'],
    max_grad_norm=params['max_grad_norm'],
    policy_kwargs=policy_kwargs,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    verbose=1,
)

print(f"\nTraining for {TOTAL_TIMESTEPS:,} timesteps...")
start_time = datetime.now()

model.learn(total_timesteps=TOTAL_TIMESTEPS, progress_bar=True)

training_time = datetime.now() - start_time
print(f"\nTraining completed in {training_time}")

# Save model
os.makedirs('models', exist_ok=True)
model.save('models/ppo_enhanced_tuned')
print("Model saved to models/ppo_enhanced_tuned.zip")

In [ ]:
# Evaluate tuned model
test_env = EnhancedPortfolioEnv(
    df=test_df_indexed,
    stock_dim=feature_info['n_tickers'],
    tech_indicator_list=feature_info['tech_indicators'],
    sentiment_feature_list=feature_info['sentiment_features'],
    include_sentiment=True,
    normalize_obs=False,
    print_verbosity=0,
    **env_config,
)

obs, _ = test_env.reset()
done = False

while not done:
    action, _ = model.predict(obs.reshape(1, -1), deterministic=True)
    obs, _, done, _, _ = test_env.step(action[0])

stats = test_env.get_portfolio_stats()

print("\n" + "="*60)
print("TUNED MODEL - TEST RESULTS")
print("="*60)
print(f"Final Portfolio Value: ${stats['final_value']:,.2f}")
print(f"Total Return: {stats['total_return']*100:.2f}%")
print(f"Sharpe Ratio: {stats['sharpe_ratio']:.3f}")
print(f"Max Drawdown: {stats['max_drawdown']*100:.2f}%")
print(f"Total Trades: {stats['total_trades']}")

# Compare with v2 baseline
print("\n" + "="*60)
print("COMPARISON WITH V2 (lr=3e-4, batch=128)")
print("="*60)
v2_sharpe = 1.221
v2_return = 41.12
print(f"V2 Sharpe: {v2_sharpe:.3f} -> Tuned: {stats['sharpe_ratio']:.3f} ({stats['sharpe_ratio']-v2_sharpe:+.3f})")
print(f"V2 Return: {v2_return:.2f}% -> Tuned: {stats['total_return']*100:.2f}% ({stats['total_return']*100-v2_return:+.2f}%)")

## 5. Download Results

In [ ]:
# Create zip of all results
!zip -r /workspace/hyperparameter_tuning_results.zip experiments/optuna_study/ models/ppo_enhanced_tuned.zip

print("\nDownload: /workspace/hyperparameter_tuning_results.zip")
print("Use RunPod File Browser or SCP")

print("\n" + "="*50)
print("REMEMBER: Stop your RunPod instance to avoid charges!")
print("="*50)